<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 7. 评估
恭喜您完成了今天的课程！希望您在此过程中学到了一些有价值的技能。现在该测试一下这些技能了。在此评估中，您将训练一种能够识别新鲜和腐烂水果的新模型。您需要使模型的验证准确率达到92％，才能通过评估，但我们鼓励您挑战更高的准确率。为此，您将使用先前练习中学到的技能，具体来说，我们建议您结合使用迁移学习、数据扩充和模型微调。训练好模型并在测试数据集上的准确率达到至少92％之后，请保存模型，然后评估其准确率。让我们开始吧！

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [2]:
def set_seed(seed):
    torch.manual_seed(seed)  # cpu 为CPU设置种子用于生成随机数，以使得结果是确定的
    torch.cuda.manual_seed(seed)  # gpu 为当前GPU设置随机种子
    torch.backends.cudnn.deterministic = True  # cudnn

set_seed(42)

## 7.1 数据集
在本练习中，您将训练一个模型来识别新鲜和腐烂的水果，数据集来自 [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)。如果您有兴趣在课后自己开始一个新的项目，那么Kaggle是一个值得访问的好地方。现在您可详细查看 `data/fruits` 文件夹中的数据集结构。水果有六类：新鲜的苹果，新鲜的橙子，新鲜的香蕉，烂的苹果，烂的橙子和烂的香蕉。这意味着您的模型将需要有6个神经元的输出层才能成功进行分类 您还需要使用 `categorical_crossentropy` 作为损失函数来编译模型，因为我们有两个以上的类别。

<img src="./images/fruits.png" style="width: 600px;">

## 7.2 加载 ImageNet 预训练的基础模型
我们鼓励您从在 ImageNet 上预训练的模型开始。您需要用正确的权重加载模型，设置输入的形状，然后选择删除模型的最后一层。请记住，图像具有三个维度：高度和宽度以及多个颜色通道。因为这些图片是彩色的，所以会有红色，绿色和蓝色三个通道。我们已经为您填写了输入形状，请不要更改，否则评估将失败。如果您需要预训练模型的参考设置，请查看 [notebook 05b](05b_presidential_doggy_door.ipynb)，您在那里最先实现的迁移学习。

In [3]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

## 7.3 冻结基础模型
接下来，我们建议您像在 [notebook 05b](05b_presidential_doggy_door.ipynb) 中一样冻结基础模型。这样做是为了使从 ImageNet 数据集中所学到的知识都不会在初始的训练中被破坏。

In [4]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

## 7.4 向模型添加新层
现在该向预训练模型中添加新层了。您可以再次使用 [notebook 05b](05b_presidential_doggy_door.ipynb) 作为指导。请密切注意最后的全连接（Dense）层，并确保其具有正确数量的神经元以对不同类型的水果进行分类。

模型的后面几层会变得更特定于训练数据。由于我们想要 VGG 进行更通用的学习，我们可以像这样选择其中的一部分:

In [5]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

拿到 VGG16 中我们需要的部分之后，就可以进行定制了。无论我们要添加什么模块，最终都要为每类输出留一个值。

In [6]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 7.5 编译模型
现在可以使用损失函数（loss）和衡量标准（metrics）选项来编译模型了。这里有 6 个类别，所以应该用什么损失函数？

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

## 7.6 扩充数据

我们将使用 VGG16 `weights` 中的变换方法来处理输入的图片。

In [8]:
pre_trans = weights.transforms()


如果需要，请尝试扩充数据以改进数据集。请参考 [notebook 04a](04a_asl_augmentation.ipynb) 和 [notebook 05b](05b_presidential_doggy_door.ipynb) 中的数据扩充的示例。您也可以查看 [TorchVision Transforms class](https://pytorch.org/vision/stable/transforms.html) 的文档。

**提示**：记住不要过度增强数据集。

In [9]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
     transforms.RandomRotation(33),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.8, 1), ratio=(1, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=.4, contrast=.4, saturation=.3, hue=.3)
])

## 7.7 加载数据集
现在应该加载训练和验证数据集了。

In [10]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"] 
    
class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

根据我们是在 `train` 还是 `valid`，选择批大小 `n`，并将 `shuffle` 相应地设置为 `True` 或 `False`。作为参考，请查看 [notebook 05b](05b_presidential_doggy_door.ipynb)。

In [11]:
n = 32
train_path = "data/fruits/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

## 7.8 训练模型
现在开始训练模型！我们已经将 `train` 和 `validate` 函数放到 [utils.py](./utils.py) 文件里了。在运行下方代码之前，确保所有变量都已正确定义。

In [ ]:
epochs = 50

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    acc = utils.validate(my_model, valid_loader, valid_N, loss_function)
    if acc >= 0.92:
        break

Epoch: 0
Train - Loss: 33.9388 Accuracy: 0.6591
Valid - Loss: 16.1167 Accuracy: 0.7325
Epoch: 1
Train - Loss: 20.3971 Accuracy: 0.7792
Valid - Loss: 7.3643 Accuracy: 0.8328
Epoch: 2
Train - Loss: 21.3583 Accuracy: 0.7707
Valid - Loss: 5.3715 Accuracy: 0.8693
Epoch: 3
Train - Loss: 15.1294 Accuracy: 0.8325
Valid - Loss: 6.0911 Accuracy: 0.8602
Epoch: 4
Train - Loss: 18.1079 Accuracy: 0.8054
Valid - Loss: 6.5543 Accuracy: 0.8419
Epoch: 5
Train - Loss: 16.0488 Accuracy: 0.8384
Valid - Loss: 9.2746 Accuracy: 0.7903
Epoch: 6
Train - Loss: 16.2545 Accuracy: 0.8325
Valid - Loss: 6.3751 Accuracy: 0.8541
Epoch: 7
Train - Loss: 13.6813 Accuracy: 0.8596
Valid - Loss: 6.0133 Accuracy: 0.8693
Epoch: 8
Train - Loss: 13.8244 Accuracy: 0.8511
Valid - Loss: 5.4065 Accuracy: 0.8723
Epoch: 9


## 7.9 解冻模型以进行微调
如果您已经达到了92％的验证准确率，则此步骤是可选的。如果没有，我们建议您以很小的学习率尝试对模型进行微调。

In [ ]:
# Unfreeze the base model
# vgg_model.requires_grad_(True)
# optimizer = Adam(my_model.parameters(), lr=.0001)

In [ ]:
# epochs = 50

# for epoch in range(epochs):
#     print('Epoch: {}'.format(epoch))
#     utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
#     utils.validate(my_model, valid_loader, valid_N, loss_function)

## 7.10 评估模型

希望您现在拥有的模型具有92％或更高的验证准确率。如果没有，您可能需要返回并对模型进行更多的训练，或者对数据增强进行调整。

对验证集准确率满意后，您可以通过执行以下单元格来评估模型。`evaluate` 函数将返回一个元组（tuple），其中第一个值是损失，第二个值是准确率。您需要获得 `0.92 或更高` 的准确率。

In [ ]:
utils.validate(my_model, valid_loader, valid_N, loss_function)

## 7.11 执行评估

请执行以下 2 个代码单元来评估您的结果。

**注意：** `run_assessment` 假设您的模型是以 `my_model` 命名的。无论出于什么原因您修改了上述名字，请在下面的单元中对 `run_assessment` 的参数做相应的修改。

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(my_model)

## 7.12 生成证书

如果您通过了评估，请返回课程页面（见下图）并单击 ASSESS TASK（评估）按钮，这样您就能获得本课程的证书了。

<img src="./images/assess_task.png" style="width: 800px;">